In [1]:
!pip install librosa pretty_midi
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install miditok
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=42ab8de18499ddd4a31c36cfab7bf474c10b9dd6408c0ee9ba2cde5de063d38d
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi
Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 124.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 

In [2]:
import os
import torch
import librosa
import pretty_midi
import numpy as np
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd
import miditok
from miditok import TokSequence
from tqdm.auto import tqdm

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [4]:
TOKENIZER_PARAMS = {
"pitch_range": (20, 110),
"beat_res": {(0,4):24, (4,12):12},
"special_tokens": ["PAD", "BOS", "EOS", "MASK"],
"use_chords": True,
"use_rests": False,
"use_tempos": True,
"use_time_signatures": True,
"use_programs": False,
"use_microtiming": False,
"ticks_per_quarter": 220,
"max_microtiming_shift": 0.125,
"num_microtiming_bins": 30,
}
config = miditok.TokenizerConfig(**TOKENIZER_PARAMS)

tokenizer=miditok.PerTok(config)

In [5]:
drive.mount('/content/drive')
DATA_ROOT='/content/drive/My Drive/song2cover'
PAD_TOKEN = tokenizer.pad_token_id
EOS_TOKEN = tokenizer.special_tokens_ids[2]
VocabSize = tokenizer.vocab_size

Mounted at /content/drive


In [6]:
def collate_fn(batch):
  xs, ys, zs, a, r = zip(*batch)



  y_max_len = max([y.shape[0] for y in ys])

  x_max_len = max([x.shape[2] for x in xs])

  y_padded = []
  for y in ys:
    pad_amt = y_max_len - y.shape[0]
    padded = F.pad(y, (0, pad_amt), value=PAD_TOKEN)
    y_padded.append(padded)

  x_padded = []
  for x in xs:
    pad_amt = x_max_len - x.shape[2]
    padded = F.pad(x, (0, pad_amt), value=0)
    x_padded.append(padded)

  x_batch = torch.stack(x_padded, dim=0)
  y_batch = torch.stack(y_padded, dim=0)
  z_batch = torch.stack(zs, dim=0)
  a_batch = torch.stack(a, dim=0)
  r_batch = torch.stack(r, dim=0)

  return x_batch, y_batch, z_batch, a_batch, r_batch

In [7]:
class AudioMidiDataset(Dataset):
  def __init__(self, root_dir, tokenizer, transform=None):
    self.root_dir = root_dir
    self.transform = transform
    df = pd.read_csv(os.path.join(root_dir, 'manifest.csv'))
    self.items = df.to_dict('records')
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.items)

  def __getitem__(self, idx):
    item = self.items[idx]
    audio_path = os.path.join(self.root_dir, item['audio_filepath'])
    midi_path = os.path.join(self.root_dir, item['midi_filepath'])
    diff = torch.tensor(float(item['difficulty'])).float()
    pos_rel = torch.tensor(float(item['pos_percent'])).float()
    pos_abs = torch.tensor(float(item['pos_absolute']), dtype=torch.long)

    y, sr = librosa.load(audio_path, sr=44100)
    stft = librosa.stft(y, n_fft=1024, hop_length=256)
    log_stft = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    x = torch.tensor(log_stft).unsqueeze(0).float()

    tokens = self.tokenizer.encode(midi_path)
    y_tokens = torch.flatten(torch.tensor(tokens, dtype=torch.long))

    eos = torch.tensor([self.tokenizer.special_tokens_ids[2]], dtype=torch.long)
    y_tokens = torch.cat((y_tokens, eos))

    sample = {
        'x': x,
        'y': y_tokens,
        'diff': diff,
        'pos_percent': pos_rel,
        'pos_absolute': pos_abs,
    }

    if self.transform:
      sample = self.transform(sample)

    return x, y_tokens, diff, pos_abs, pos_rel

In [8]:
dataset = AudioMidiDataset(DATA_ROOT, tokenizer)

loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

In [9]:
class AudioEncoder(nn.Module):
  def __init__(self, d_model=512):
    super().__init__()

    self.conv = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3, stride=(2,2), padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3, stride=(2,2), padding=1),
        nn.ReLU(),
        nn.Conv2d(128, d_model, kernel_size=3, stride=(2,2), padding=1),
        nn.ReLU()
    )

    self.project = nn.Linear(in_features=d_model * 65, out_features=d_model)
    self.pos_enc = nn.Embedding(num_embeddings=1500, embedding_dim=d_model)

    enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=8, dim_feedforward=d_model * 4)
    self.encoder = nn.TransformerEncoder(enc_layer, num_layers=3)

  def forward(self, x):
    z = self.conv(x)

    B, D, F_enc, T_enc = z.shape

    enc_in = z.permute(0, 3, 1, 2).reshape(B, T_enc, D * F_enc)
    enc_proj = self.project(enc_in)

    positions = torch.arange(T_enc, device=x.device).unsqueeze(0)
    enc_pos = enc_proj + self.pos_enc(positions)

    enc_in_tp = enc_pos.permute(1,0,2)
    enc_out_tp = self.encoder(enc_in_tp)

    enc_out = enc_out_tp.permute(1,0,2)

    return enc_out

class TokenDecoder(nn.Module):
  def __init__(self, vocab_size, d_model=512, max_len=1500, max_measures=200,):
    super().__init__()


    self.d_model = d_model

    self.token_emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model)
    self.pos_emb = nn.Embedding(num_embeddings=max_len, embedding_dim=d_model)
    self.measure_emb = nn.Embedding(num_embeddings=max_measures, embedding_dim=d_model)
    self.diff_proj = nn.Sequential(
        nn.Linear(in_features=1, out_features=d_model),
        nn.ReLU(),
        nn.Linear(d_model, d_model)
    )

    self.rel_proj = nn.Sequential(
        nn.Linear(in_features=1, out_features=d_model),
        nn.ReLU(),
        nn.Linear(d_model, d_model)
    )

    dec_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=8, dim_feedforward=d_model * 4)
    self.decoder = nn.TransformerDecoder(dec_layer, num_layers=3)

    self.output_fc = nn.Linear(d_model, vocab_size)

  def forward(self, y_input, memory, diff, pos_abs, pos_rel):
    B, T_tgt = y_input.shape

    tok_embed = self.token_emb(y_input)

    pos = torch.arange(T_tgt, device = y_input.device).unsqueeze(0)
    pos_embed = self.pos_emb(pos)

    measure_emb = self.measure_emb(pos_abs)
    measure_b = measure_emb.unsqueeze(1).expand(-1, T_tgt, -1)

    diff = diff.unsqueeze(-1)
    diff_emb = self.diff_proj(diff)
    diff_b = diff_emb.unsqueeze(1).expand(-1, T_tgt, -1)

    rel = pos_rel.unsqueeze(-1)
    rel_emb = self.rel_proj(rel)
    rel_b = rel_emb.unsqueeze(1).expand(-1, T_tgt, -1)

    tgt = tok_embed + pos_embed + measure_b + diff_b + rel_b

    tgt_mask = nn.Transformer.generate_square_subsequent_mask(T_tgt).to(y_input.device)

    tgt2 = tgt.permute(1,0,2)
    mem2 = memory.permute(1,0,2)

    dec_out2 = self.decoder(tgt2, mem2, tgt_mask=tgt_mask)

    dec_out = dec_out2.permute(1,0,2)

    logits = self.output_fc(dec_out)

    return logits


class Audio2Midi(nn.Module):
  def __init__(self, vocab_size, d_model=512, max_tok_len=1500):
    super().__init__()
    self.encoder = AudioEncoder(d_model=d_model)
    self.decoder = TokenDecoder(vocab_size, d_model=d_model, max_len=max_tok_len)

  def forward(self, x_audio, y_input, diff, pos_abs, pos_rel):
    memory = self.encoder(x_audio)
    logits = self.decoder(y_input, memory, diff, pos_abs, pos_rel)

    return logits



Add `%load_ext cudf.pandas` before importing pandas to speed up operations using GPU

In [10]:
model = Audio2Midi(vocab_size=VocabSize, d_model=512, max_tok_len=1500).to(device)
optim = torch.optim.AdamW(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()

criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)

num_epochs = 10

for epoch in range(1, num_epochs+1):
  model.train()
  total_loss = 0.0

  batch_iterator = tqdm(loader, desc=f"Epoch {epoch:02d}", leave=False, bar_format='{l_bar}{bar} | {percentage:3.0f}%')

  for x_batch, y_batch, diff_batch, abs_batch, rel_batch in batch_iterator:
      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)
      diff_batch = diff_batch.to(device)
      abs_batch = abs_batch.to(device)
      rel_batch = rel_batch.to(device)

      y_input  = y_batch[:, :-1]
      y_target = y_batch[:, 1:]

      optim.zero_grad()

      with torch.cuda.amp.autocast():
        logits = model(x_batch, y_input, diff_batch, abs_batch, rel_batch)

        B, Tm, V = logits.shape
        loss = criterion(
            logits.reshape(-1, V),
            y_target.reshape(-1),
            ignore_index=PAD_TOKEN
        )

      scaler.scale(loss).backward()
      scaler.step(optim)
      scaler.update()

      total_loss += loss.item()
      batch_iterator.set_postfix(loss=f"{loss.item():.4f}")

  avg_loss = total_loss / len(loader)
  print(f"Epoch {epoch:02d} — Loss {avg_loss:.4f}")

torch.save({
  'epoch': epoch,
  'model_state_dict': model.state_dict(),
  'optimizer_state_dict': optim.state_dict()}, os.path.join(DATA_ROOT, 'checkpoint.pt'))


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 01:   0%|           |   0%

Epoch 01 — Loss 3.1996


Epoch 02:   0%|           |   0%

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.40 GiB. GPU 0 has a total capacity of 39.56 GiB of which 1.44 GiB is free. Process 20617 has 38.11 GiB memory in use. Of the allocated memory 11.91 GiB is allocated by PyTorch, and 25.70 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
torch.save(model.state_dict(), os.path.join(DATA_ROOT, 'model.pt'))

In [ ]:
model.load_state_dict(torch.load(os.path.join(DATA_ROOT, 'model.pt'), map_location=device))

<All keys matched successfully>

In [ ]:
def generate_midi(model, tokenizer, audio_path, diff, pos_abs, pos_rel, max_len=1300, device='cpu', temp = 1.0):
  model.eval()

  y, sr = librosa.load(audio_path, sr=44100)
  stft = librosa.stft(y, n_fft=1024, hop_length=256)
  log_stft = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
  x = torch.tensor(log_stft, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

  diff = torch.tensor([diff]).float().to(device)
  pos_rel = torch.tensor([pos_rel]).float().to(device)
  pos_abs = torch.tensor([pos_abs], dtype=torch.long).to(device)


  with torch.no_grad():
    memory = model.encoder(x)

  generated = [tokenizer.special_tokens[1]]
  for _ in range(max_len):
    if _ % 100 == 0:
      print(_)
    y_input = torch.tensor([tokenizer.vocab[t] for t in generated], dtype=torch.long).unsqueeze(0).to(device)
    with torch.no_grad():
      logits = model.decoder(y_input, memory, diff, pos_abs, pos_rel)
    next_logits = logits[:, -1, :]
    probabilities = F.softmax(next_logits / temp, dim=-1)
    next_id = torch.multinomial(probabilities, num_samples=1).squeeze(0).item()
    next_token = [key for key, val in tokenizer.vocab.items() if val == next_id][0]
    generated.append(next_token)
    if next_id == tokenizer.special_tokens_ids[2]:
      break

  if generated[0] == tokenizer.special_tokens[1]:
    generated = generated[1:]
  if generated[-1] == tokenizer.special_tokens[2]:
    generated = generated[:-1]
  generated_seq = TokSequence(tokens=generated)
  print(len(generated_seq))

  tokens = tokenizer.decode([generated_seq])
  return tokens


In [ ]:
path = os.path.join(DATA_ROOT,"song", "0_0.mp3")


tokens = generate_midi(model, tokenizer, path, 1.5, 40, 0.4, max_len=2000, device=device, temp=1.0)

0
100
200
300
381


In [ ]:
print(tokens)

tokens.dump_midi(os.path.join(DATA_ROOT, '1.mid'))

Score(ttype=Tick, tpq=220, begin=0, end=13847, tracks=1, notes=101, time_sig=1, key_sig=0, markers=0)
